In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import textblob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from xgboost import XGBClassifier

In [2]:
def analize_sentiment(tweet):
    
    analysis = TextBlob((str(tweet)))     #defining the function which will find the plority of a sentence
    return analysis.polarity 

In [3]:
news = pd.read_csv(r'C:\Users\HARSHIT\Downloads\Combined_News_DJIA.csv\Combined_News_DJIA.csv')

In [4]:
news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [5]:
train_news = news[news['Date'] < '2014-07-15']   # SPLITTING THE DATASET INTO TRAINING AND TESTING
test_news = news[news['Date'] > '2014-07-14']

In [6]:
train_news_list = []
for row in range(0,len(train_news.index)): # CONVERT THE TRAINNG DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
    train_news_list.append(' '.join(str(k) for k in train_news.iloc[row,2:27]))

In [7]:
vectorize= CountVectorizer(min_df=0.01, max_df=0.8) # DEFINING THE VECTOR FUNCTION, SPECIFYING THR MIN AND MAX WORD FREQUENCY FILTER
news_vector = vectorize.fit_transform(train_news_list) # TRANSFORMING THE TRAINING DATASET INTO WORD FREQUENCY TRANFORMATION
print( "THE TABLE OF FREQUENCY WORD DISTRIBUTION" , news_vector.shape)

THE TABLE OF FREQUENCY WORD DISTRIBUTION (1492, 4733)


In [8]:
lr=LogisticRegression()
model = lr.fit(news_vector, train_news["Label"])
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))# CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY

test_vector = vectorize.transform(test_news_list) # TRANSFORMING THE TESTING DATASET INTO WORD FREQUENCY TRANFORMATION

predictions = model.predict(test_vector)

pd.crosstab(test_news["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

accuracy1=accuracy_score(test_news['Label'], predictions)
print("the baseline model accuracy", accuracy1)

words = vectorize.get_feature_names()
coefficients = model.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : words,'Coefficient' : coefficients})  # WORD DISTRIBUTION OF THE MODEL

coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print("Top ten words according to the baseline model",coeffdf.head(10))
print("Last ten words according to the baseline model",coeffdf.tail(10))

C:\Users\HARSHIT\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


the baseline model accuracy 0.4607645875251509
Top ten words according to the baseline model           Word  Coefficient
3728      self     0.627790
4647      wing     0.532141
2090  hospital     0.531786
2392     kills     0.524604
284      among     0.516700
4387      turn     0.515006
762     cartel     0.513382
2929  olympics     0.509177
1146  damascus     0.505388
3585      rise     0.503335
Last ten words according to the baseline model            Word  Coefficient
3770        sex    -0.540523
1163         de    -0.540868
990       congo    -0.548510
4206     terror    -0.555358
4047   students    -0.563924
3653  sanctions    -0.569133
2100      hours    -0.572312
506       begin    -0.602409
4301      total    -0.610861
3626        run    -0.665484


In [9]:
nvectorize = TfidfVectorizer(min_df=0.05, max_df=0.85,ngram_range=(2,2)) # DEFINING THE TFID TRANSFORMATION FUNCTION
news_nvector = nvectorize.fit_transform(train_news_list)

print(" TFID TRANSFOMATION DATAFRAME SHAPE",news_nvector.shape)

 TFID TRANSFOMATION DATAFRAME SHAPE (1492, 284)


In [10]:
nmodel = lr.fit(news_nvector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
ntest_vector = nvectorize.transform(test_news_list)
npredictions = nmodel.predict(ntest_vector)

pd.crosstab(test_news["Label"], npredictions, rownames=["Actual"], colnames=["Predicted"])

accuracy2=accuracy_score(test_news['Label'], npredictions)
print(" Logistics Regression with Bigram and TFID",accuracy2)

nwords = nvectorize.get_feature_names()
ncoefficients = nmodel.coef_.tolist()[0]
ncoeffdf = pd.DataFrame({'Word' : nwords, 
                        'Coefficient' : ncoefficients})
ncoeffdf = ncoeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
ncoeffdf.head(10)
ncoeffdf.tail(10)

 Logistics Regression with Bigram and TFID 0.5311871227364185


,Word,Coefficient
242,to help,-0.870909
262,us and,-0.880771
4,accused of,-0.952199
153,people are,-0.974838
20,around the,-1.009906
175,south africa,-1.019293
6,after being,-1.088850
244,to kill,-1.197320
188,the country,-1.225612
260,up in,-1.273317


In [11]:
nvectorize = TfidfVectorizer(min_df=0.01, max_df=0.95,ngram_range=(2,2))
news_nvector = nvectorize.fit_transform(train_news_list)

rfmodel = RandomForestClassifier(random_state=55)  #DEFINNG THE RANDOM FOREST MODEL
rfmodel = rfmodel.fit(news_nvector, train_news["Label"])
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))
ntest_vector = nvectorize.transform(test_news_list)

rfpredictions = rfmodel.predict(ntest_vector)
accuracyrf = accuracy_score(test_news['Label'], rfpredictions)
print("Random forest with tfid and bigram", accuracyrf)

Random forest with tfid and bigram 0.5372233400402414


In [12]:
nvectorize = TfidfVectorizer(min_df=0.05, max_df=0.8,ngram_range=(2,2)) #DEFINING THE NAIVE BAYS MODEL
news_nvector = nvectorize.fit_transform(train_news_list)

nbmodel = MultinomialNB(alpha=0.5)
nbmodel = nbmodel.fit(news_nvector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
ntest_vector = nvectorize.transform(test_news_list)

nbpredictions = nbmodel.predict(ntest_vector)
nbaccuracy=accuracy_score(test_news['Label'], nbpredictions)
print("Naive Bayes accuracy: ",nbaccuracy)

Naive Bayes accuracy:  0.5291750503018109


In [13]:
gbmodel = GradientBoostingClassifier(random_state=52)  # DEFINING THE GARDIANT BOOSTING MODEL
gbmodel = gbmodel.fit(news_nvector, train_news["Label"])
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))
ntest_vector = nvectorize.transform(test_news_list)

gbpredictions = gbmodel.predict(ntest_vector.toarray())
gbaccuracy = accuracy_score(test_news['Label'], gbpredictions)

from sklearn.metrics import confusion_matrix
print(" CONFUSION MATRIX OF THE GRADIANT BOOSTING ", confusion_matrix(test_news['Label'], gbpredictions))


print("Gradient Boosting accuracy: ",gbaccuracy)

 CONFUSION MATRIX OF THE GRADIANT BOOSTING  [[ 92 148]
 [ 74 183]]
Gradient Boosting accuracy:  0.5533199195171026


In [14]:
n3vectorize = TfidfVectorizer(min_df=0.0004, max_df=0.115,ngram_range=(3,3)) # DEFINING THE TFID , TRIGRAM MODEL
news_n3vector = n3vectorize.fit_transform(train_news_list)

print(news_n3vector.shape)

n3model = lr.fit(news_n3vector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
n3test_vector = n3vectorize.transform(test_news_list)
n3predictions = n3model.predict(n3test_vector)

pd.crosstab(test_news["Label"], n3predictions, rownames=["Actual"], colnames=["Predicted"])

accuracy3=accuracy_score(test_news['Label'], n3predictions)
print("TRIGARAM ACCURACY", accuracy3)

(1492, 569061)
TRIGARAM ACCURACY 0.5171026156941649


In [15]:
n3words = n3vectorize.get_feature_names()
n3coefficients = n3model.coef_.tolist()[0]
n3coeffdf = pd.DataFrame({'Word' : n3words, 
                        'Coefficient' : n3coefficients})
n3coeffdf = n3coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print("trigram top ten word distibution", n3coeffdf.head(10))
print("trigram last ten word distibution", n3coeffdf.tail(10))    # trigram model word distribution 

trigram top ten word distibution                       Word  Coefficient
509383           to the us     0.201353
481307        the right to     0.170757
322285   nobel peace prize     0.165954
223934  human rights watch     0.159561
491158         this is the     0.154585
240342        in west bank     0.151582
230935        in china the     0.139373
518984         turn out to     0.137981
239146     in the occupied     0.132398
321584         no fly zone     0.127243
trigram last ten word distibution                          Word  Coefficient
183898      freedom of speech    -0.141552
356524        osama bin laden    -0.142172
371338  phone hacking scandal    -0.147765
497344              to be the    -0.148226
207018      has been arrested    -0.152082
509742              to try to    -0.152849
334728        of human rights    -0.170649
416292             said to be    -0.191760
48303        around the world    -0.195609
238814         in the country    -0.223890


In [16]:
train_sentiment=train_news
test_sentiment = test_news
train_sentiment =train_sentiment.drop(['Date', 'Label'], axis=1)
for column in train_sentiment:
    train_sentiment[column]=train_sentiment[column].apply(analize_sentiment)  #converting the train headlines into polarity scores
train_sentiment = train_sentiment+10  # removing negative co:efficient from the datset for better performance

test_sentiment =test_sentiment.drop(['Date', 'Label'], axis=1)
for column in test_sentiment:
    test_sentiment[column]=test_sentiment[column].apply(analize_sentiment) # converting the test headlines into ploarity 
test_sentiment=test_sentiment+10 # removing negative co:efficient from the datset for better performance

XGB_model= XGBClassifier()  # training the polarity score datset with DIJA 
gradiant=XGB_model.fit(train_sentiment, train_news['Label'])
y_pred= gradiant.predict(test_sentiment)


from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_news['Label'], y_pred))
from sklearn.metrics import accuracy_score
print("Sentiment Accuracy",accuracy_score(test_news['Label'], y_pred))
from sklearn.metrics import f1_score
print("f1_score__",f1_score(test_news['Label'], y_pred, average='weighted'))

C:\Users\HARSHIT\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[106 134]
 [117 140]]
Sentiment Accuracy 0.4949698189134809
f1_score__ 0.49378249378558736


C:\Users\HARSHIT\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
